The following cell ensures that each notebook runs on the same dependencies and versions as all other notebooks in Google Colab.

In [ ]:
import os, sys, subprocess
from pathlib import Path

def in_colab() -> bool:
    try:
        import google.colab
        return True
    except Exception:
        return False

REQ_PATH = Path("requirements.txt")
RAW_REQ_URL = os.environ.get(
    "RAW_REQ_URL",
    "https://raw.githubusercontent.com/makinchii/CS156_Project/main/requirements.txt"
)

def run(cmd):
    print("$", " ".join(cmd))
    subprocess.check_call(cmd)

pip = [sys.executable, "-m", "pip"]

if not in_colab() and os.environ.get("VIRTUAL_ENV") is None:
    print("Not in a virtual environment. Consider: python -m venv .venv && activate it.")

# Upgrade pip in the active kernel
run(pip + ["install", "-q", "-U", "pip"])

# Install dependencies
if REQ_PATH.exists():
    run(pip + ["install", "-q", "-r", str(REQ_PATH)])
elif in_colab() and RAW_REQ_URL:
    run(pip + ["install", "-q", "-r", RAW_REQ_URL])
else:
    print("requirements.txt not found. If you're in Colab, set RAW_REQ_URL to your repo's raw requirements.txt, e.g.:")
    print("    os.environ['RAW_REQ_URL'] = 'https://raw.githubusercontent.com/<user>/<repo>/main/requirements.txt'")

# Quick sanity print
try:
    import pandas as pd, numpy as np
    print("Python:", sys.version.split()[0])
    print("pandas:", pd.__version__)
    print("numpy :", np.__version__)
except Exception as e:
    print("(Optional) Could not import pandas/numpy for version check:", e)

$ /usr/bin/python3 -m pip install -q -U pip
$ /usr/bin/python3 -m pip install -q -r https://raw.githubusercontent.com/makinchii/CS156_Project/main/requirements.txt
Python: 3.12.12
pandas: 2.2.2
numpy : 2.0.2


In [ ]:
# import packages
import pandas as pd
import numpy as np
import re

In [ ]:
from utils import load_dataset

df = load_dataset(local_rel_path="data/biosensor_dataset_with_target.csv", raw_url="https://raw.githubusercontent.com/makinchii/CS156_Project/main/data/biosensor_dataset_with_target.csv")
print(df.head())

Reading from RAW URL: https://raw.githubusercontent.com/makinchii/CS156_Project/main/data/biosensor_dataset_with_target.csv
   Heart_Rate     Acc_X     Acc_Y     Acc_Z      Gyro_X      Gyro_Y  \
0         166 -2.598157  2.205079 -0.649084  -48.457966   15.407057   
1         159 -2.862471  2.173831 -2.054184    5.137738   34.974254   
2         120 -1.045441  2.755975  2.696018 -175.960480 -107.481553   
3         119  2.931939  0.005220 -1.835028   -8.255970  131.909447   
4         121  2.517730  1.217080  1.611578  -44.266911  147.303725   

       Gyro_Z   Event_Label                   Timestamp Athlete_ID  
0  164.550534         Accel  2025-05-11 12:14:53.417747       A001  
1  -15.283653  Jump_Takeoff  2025-05-11 12:14:53.517747       A001  
2  177.757215          Stop  2025-05-11 12:14:53.617747       A001  
3 -172.149554    Sprint_Mid  2025-05-11 12:14:53.717747       A001  
4  104.186863     Start_Run  2025-05-11 12:14:53.817747       A001  
